In [1]:
import os
import ast
import pandas as pd

In [2]:
df_dron_park = pd.read_csv(r"C:\Users\user\Desktop\python 정리집\웹크롤링 팀프로젝트\df_dron_park.csv", encoding='cp949')
circle_table = pd.read_csv(r"C:\Users\user\Desktop\python 정리집\웹크롤링 팀프로젝트\Circle_table.csv", encoding='utf-8')
NE_table = pd.read_csv(r"C:\Users\user\Desktop\python 정리집\웹크롤링 팀프로젝트\NE_table.csv", encoding='utf-8')

In [3]:
df_dron_park

,지역,좌표
0,대전 금강변 드론공원,"[(36.465, 127.39055555555557), (36.46583333333..."
1,광주 북구 영산강변 드론공원,"[(35.22166666666667, 126.86166666666666)]"
2,광나루 한강변 드론공원,"[(37.546638888888886, 127.12052777777777), (12..."


In [28]:
# "드론공원"까지의 부분만 남기는 함수
def extract_location_name(region):
    if pd.isnull(region):
        return region
    if "드론공원" in region:
        return region.split("드론공원")[0] + "드론공원"
    return region

# df_dron_park.csv 파일 읽기 (인코딩은 파일에 맞게 조정)
df_dron = pd.read_csv("df_dron_park.csv", encoding="cp949")

# 좌표 컬럼이 문자열 형태로 저장되어 있다면 ast.literal_eval로 리스트로 변환
df_dron["좌표"] = df_dron["좌표"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# '지역' 컬럼의 값을 '위치' 컬럼으로 재설정 (드론공원까지의 부분만 남김)
df_dron["위치"] = df_dron["지역"].apply(extract_location_name)

# 신규 데이터를 NE와 Circle 데이터로 분리
# NE: 좌표 목록의 길이가 2 이상인 경우 (예: 대전 금강변 드론공원)
df_NE_new = df_dron[df_dron["좌표"].apply(lambda x: isinstance(x, list) and len(x) > 1)].copy()
df_NE_new["NE"] = df_NE_new["좌표"].apply(lambda x: str(x))
df_NE_new['구역 코드'] = ''
df_NE_new = df_NE_new[["구역 코드", "위치", "NE"]]
print(df_NE_new)

# Circle: 좌표 목록의 길이가 1인 경우 (예: 광주 북구 영산강변 드론공원 등)
df_Circle_new = df_dron[df_dron["좌표"].apply(lambda x: isinstance(x, list) and len(x) == 1)].copy()
# 좌표 목록의 단일 쌍에서 위도와 경도를 분리
df_Circle_new["위도"] = df_Circle_new["좌표"].apply(lambda x: x[0][0] if isinstance(x, list) and len(x)==1 else None)
df_Circle_new["경도"] = df_Circle_new["좌표"].apply(lambda x: x[0][1] if isinstance(x, list) and len(x)==1 else None)
# 만약 반경 데이터가 df_dron에 있다면 사용, 없으면 None 처리 (여기서는 None으로 처리)
df_Circle_new["반경"] = None 
df_Circle_new['구역 코드'] = ''
df_Circle_new = df_Circle_new[["구역 코드", "위치", "반경", "위도", "경도"]]
print(df_Circle_new)

# --- 최종 데이터 병합 (append) ---
final_ne = pd.merge(NE_table, df_NE_new, on = '구역 코드', how='left')
final_circle = pd.merge(circle_table, df_Circle_new, on = '구역 코드', how='left')

final_ne = final_ne.drop([final_ne.columns[3], final_ne.columns[4]], axis=1)
final_ne = final_ne.rename(columns={'위치_x':'위치', 'NE_x':'NE'})

final_circle = final_circle.drop(final_circle.columns[5:9], axis=1)
final_circle = final_circle.rename(columns={'위치_x':'위치', '반경_x':'반경', '위도_x':'위도', '경도_x':'경도'})

# 최종 데이터 출력 (확인용)
print("=== 최종 NE Table ===")
print(final_ne)
print("=== 최종 Circle Table ===")
print(final_circle)

# 최종 결과를 CSV 파일로 저장
# final_ne.to_csv("NE_table.csv", encoding="utf-8-sig", index=False)
# final_circle.to_csv("Circle_table.csv", encoding="utf-8-sig", index=False)


  구역 코드            위치                                                 NE
0         대전 금강변 드론공원  [(36.465, 127.39055555555557), (36.46583333333...
2        광나루 한강변 드론공원  [(37.546638888888886, 127.12052777777777), (12...
  구역 코드               위치    반경         위도          경도
1        광주 북구 영산강변 드론공원  None  35.221667  126.861667
=== 최종 NE Table ===
      구역 코드           위치                                                 NE
0      UA 9           양평  [(37.45, 127.38333333333334), (37.502777777777...
1     UA 14           공주  [(36.45805555555556, 126.96388888888889), (36....
2     UA 19          시화호  [(37.212500000000006, 126.67472222222223), (37...
3     UA 25           하동  [(34.98777777777778, 127.72333333333333), (35....
4     UA 26          장암산  [(37.36972222222222, 128.39194444444445), (37....
5   * UA 31           청라  [(37.5625, 126.62666666666667), (37.565, 126.6...
6   * UA 33          병천천  [(36.647777777777776, 127.34972222222221), (36...
7   * UA 34          미호천  [(36.61388888888889